In [14]:
# the following packages should be installed.
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import re
from datetime import datetime
import ast

######## USER ENTRY REQUIRED#################
# Enter the base path where you will want to save simulation data. 
# As written, this will add your simulation results to the Downloads folder 
# if your profile username is ishaan.
base_path = "/Users/ishaan/Downloads" ####USER ENTRY######## 


In [24]:
def current_datetime():
    """function simply will create a string value of the current 
    date and time so you can label you simulation results easily"""
    return datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [16]:
def calc_mRNA_sim(number_cells, load_cell_dist, params, **kwargs):
    """takes in the number of cells (without replicates) that you want to load mRNA with as well as with the loading
    distribution. Suggests a number of mRNAs to simulate
    
    Parameters
    -----------
    number_cells: int
        Number of cells (wihtout replicates) that you want to load with mRNA

    load_cell_dist: (callable)
        A np.random distribution to sample from. Size in the load distribution is the number of times you sample from distribtion. 
        **kwargs can be used to unpack parameters into the distribution of choice.

    Returns
    -------
    int
        Number of mRNAs to simulate in your class instance.
    
    """
    
    samples=load_cell_dist(*params, **kwargs)

    number_mRNA_sim = int(np.round(number_cells * (np.mean(samples) + np.std(samples))))
    return number_mRNA_sim

In [17]:
class SemperPredict:
    """
    SemperPredict is a class object that can be used to simulate many monte carlo mRNAs. Particular class methods
    can then be used to load results of these simulations into cells for estimation of transient transfection behavior.
    """
    
    def __init__(self, pTIS, number_mRNA):
        """
        Initializes the MCmRNA object.

        Parameters
        ----------
        pTIS : list
            List of floats containing the probabilities of scanning ribosome 
            initiating translation at TIS 1, TIS 2, TIS 3, and so on. 
            Accepts a list of any size

        number_mRNA : int
            number of mRNA you wish to simulate for your mRNA Array.

        """
        self.pTIS = pTIS
        self.number_mRNA = number_mRNA
        self.number_ORFs = len(pTIS)
        
        
    def sample_ribosomes(self, rib_dist_function, params, **kwargs):
        """Sample the number of ribosomes from a distribution that will traverse the mRNA in your
        simulations.
        
        Parameters
        ----------
        distribution_function: np.random distribution function
            A distribution to sample from called from np.random (i.e. np.random.normal())
            **kwargs are keyword arguments required to sample from the particular np.random distribution

        Returns
        -------
        np.array
            1xnumber_mRNA array of ribosome numbers that will traverse each mRNA in 1:number_mRNA
        
        """
        # sample the number of ribosomes that will cross each mRNA that you will simulate.
        random_ribosomes = rib_dist_function(*params, **kwargs, size = self.number_mRNA)
    
        # Round the samples to the nearest integer to discretize and ensure they are not less than 0.
        # If they are negative, set them to 0.
        # In future, one can resample to avoid negatives.
        random_ribosomes = np.maximum(np.round(random_ribosomes), 0).astype(int)
        
        self.random_ribosomes = random_ribosomes

        return 
    
    def make_mRNA_array(self):
        """Create an array of the results of each monte carlo mRNA simulation

        Parameters:
        mc_sim: callable
            The monte carlo simulation method you want to use to simulate each mRNA.
        """

        #initialize array to save the protein counts from each simulation.
        mRNA_array = np.zeros((self.number_mRNA, self.number_ORFs))
    
        for i, r in enumerate(self.random_ribosomes):
            mRNA_array[i, :] = self.simulate_MCmRNA(r)
        
        self.mRNA_array = mRNA_array
        return 
    
    
    def simulate_MCmRNA(self, trav_rib):
        """ Function will take each ribosome in trav_rib and will model the ribosome traversing the mRNA. 
        The ribosome considers sequential Boolean conditions to mimic a ribosome moving from 5' to 3' as it 
        scans for TISs and successfully or unsuccessfully initiates translation.
        
        Parameters
        ----------
        trav_rib: int
            Number of ribosomes to simulate that will traverse the mRNA
        
        Returns
        -------

        np.array
            2x1 array of with total amount of protein 1 and protein 2 made.
        """
        
        proteins = np.zeros([1, self.number_ORFs])
        
        # for every r ribosome, simulate if the first, second, third, none, etc. ORF
        # is successfully translated.
        for r in range(0, trav_rib):
            
            # for each value in pTIS (indicative of TIS in front of ORF from 5'-->3')
            # sample from a Bernoulli distribution using the success probability pTIS_prob
            for i, pTIS_prob in enumerate(self.pTIS):
                
                if np.random.uniform(low=0, high=1) <= pTIS_prob:
                    proteins[0,i] += 1
                    break # if criteria met, simulate next ribosome

        return proteins
    
    
    def load_cells(self, number_cells, number_replicates, describe, load_cell_dist, params, **kwargs):
        """Given the number of cells needed to be predicted, the function will 
        take in an mRNA_array and fill the cells with j mRNA where j is a number sampled from a normal distribution
        
        Parameters
        ----------
        number_cells: int
            The number of cells you want to load with mRNA. You can increase the number of replicates
            to resample the simulated mRNAs and increase this number without having to simulate new mRNA

        number_replicates: int
            Number of replicates to produce by resampling the mRNA array

        describe: str
            String describing the type of gene circuit being tested.

        load_cell_dist: (callable)
            Np.random distribution function for sampling number of mRNA results to go into each cell. 
            params and kwargs can be used to specify parameters for the distribution.

        Returns
        -------
        pd.DataFrame
            Realizations of predicted protein quantities for single cells.
        """

        cell_array = []

        for r in range(0, number_replicates):
            # randomize the mRNA_array between iterations
            mRNA_array_rep = np.copy(self.mRNA_array)
            np.random.shuffle(mRNA_array_rep)

            # sample the number of mRNA results to load into each cell
            random_mRNA = load_cell_dist(*params, **kwargs, size = number_cells)
            
            # round the number of mRNA to the nearest integer and make sure the value is positive, 
            # if negative set the value to 0. This can be improved in the future by resampling negative values.
            random_mRNA = np.maximum(np.round(random_mRNA), 0).astype(int)
            
            # load m mRNAs into a cell, and then delete them from the shuffled array of mRNA simulation results.
            for i, m in enumerate(random_mRNA):
                cell_array.append(mRNA_array_rep[0:m,:].sum(axis=0))
                mRNA_array_rep = np.delete(mRNA_array_rep, slice(0,m), axis=0)
                
                
        output_df = pd.DataFrame(cell_array)
        output_df["description"] = describe
        
        self.simulated_cells = output_df
        return 
        
                

# Internal Methionine Comparisons Example

In [26]:
# metadata definitions for example simulation.
internal_met_test_params = pd.DataFrame([{'no_cells': 1000,
 'no_reps': 4,
 'no_orfs': 3,
 'load_dist': 'np.random.normal',
 'load_params': '[100, 100]',
 'rib_dist': 'np.random.normal',
 'rib_params': '[100,100]',
 'pnostrt': 0.0001,
 'pTTT': 0.1,
 'pCCC': 0.5,
 'pACC': 0.9,
 'pempty': 0}])

In [ ]:
# The following code will unpack the meta data and start two simulations. 
# The simulations compare what will happen when an internal methionine exists in the first ORF.
# It will make graphs and save them to the folder in base_path.

# Unpack metadata.
for index, rows in internal_met_test_params.iterrows():
  date_time_meta = current_datetime()
  no_cells = int(rows.no_cells)
  no_reps = int(rows.no_reps)
  no_orfs = int(rows.no_orfs)
  load_dist = eval(rows.load_dist)
  load_params = ast.literal_eval(rows.load_params)
  rib_dist = eval(rows.rib_dist)
  rib_params = ast.literal_eval(rows.rib_params)
  number_mRNA_calc = calc_mRNA_sim(no_cells, load_dist, load_params, size=5000)
  pnostrt=float(rows.pnostrt)
  pTTT= float(rows.pTTT)
  pCCC= float(rows.pCCC)
  pACC= float(rows.pACC)
  pempty = float(rows.pempty)

  # define the directory name where your results will be saved.
  simulation_name = "simulation_" + date_time_meta +"_" + re.findall("method ([\d\w]{0,})", str(load_dist))[0] +"_" + str(load_params[0]) +"_" +str(load_params[1]) +"_" + re.findall("method ([\d\w]{0,})", str(rib_dist))[0] + "_" + str(rib_params[0]) + "_" + str(rib_params[1]) +"_" + str(no_cells) + "cells_" + str(no_reps) + "reps"

  # define the directory path where your results will be saved.
  simulation_path = base_path + "/" + simulation_name

  #terminal command to create the directiory simulation_path
  !mkdir $simulation_path

  # save the meta data for your simulation so you can refer back to it if needed.
  meta = pd.DataFrame({"date_time_meta" : date_time_meta,
              "no_cells" : no_cells,
            "no_reps": no_reps,
            "no_orfs" : no_orfs,
            "load_dist" : re.findall("method ([\d\w]{0,})", str(load_dist))[0],
            "load_params": [load_params],
            "rib_dist" : re.findall("method ([\d\w]{0,})", str(rib_dist))[0],
            "rib_params" :[rib_params],
            "number_mRNA_calc" :number_mRNA_calc,
            "pnostrt":pnostrt,
            "pTTT":pTTT,
            "pCCC":pCCC,
            "pACC":pACC,
            "pempty":pempty})

  meta.to_csv(simulation_path + "/" + date_time_meta + "_meta_data.csv")

  # Example simulation where we do not have an internal methionine. Therefore, the second pTIS is pEmpty = 0. TTT/ACC is SEMPER combo used.
  TTT_ACC_ins = SemperPredict(pTIS=[pTTT,pempty, pACC],  number_mRNA=number_mRNA_calc)
  TTT_ACC_ins.sample_ribosomes(rib_dist, rib_params)
  TTT_ACC_ins.make_mRNA_array()
  TTT_ACC_ins.load_cells(no_cells, no_reps, 'TTT/ACC', load_dist, load_params)

  # Example simulation where we do have an internal methionine. The internal methionine has an approximate strength of CCC
  # Therefore we use "pCCC" which will simulate an internal methionine with TIS probability of ~pCCC=0.5. TTT/ACC is SEMPER combo used.
  TTT_immut_ACC_ins = SemperPredict(pTIS=[pTTT,pCCC, pACC],  number_mRNA=number_mRNA_calc)
  TTT_immut_ACC_ins.sample_ribosomes(rib_dist, rib_params)
  TTT_immut_ACC_ins.make_mRNA_array()
  TTT_immut_ACC_ins.load_cells(no_cells, no_reps, 'TTT/immutable/ACC', load_dist, load_params)

  # combine your results into one dataframe
  sim_results = pd.concat([TTT_ACC_ins.simulated_cells, TTT_immut_ACC_ins.simulated_cells], ignore_index=True)

  # save your results
  sim_results.to_csv(simulation_path + "/" + date_time_meta + "_results.csv")

  # generate and save a scatter plot
  scatter = sns.scatterplot(x=0, y=2, data=sim_results, hue = 'description')
  plt.xscale('log')
  plt.yscale('log')

  scatter.get_figure().savefig(simulation_path + "/" + date_time_meta + "_scatterplot_data.png")
  plt.close()


  # generate and save distribution plots for no internal methionine
  TTT_ACC = sns.displot(data=sim_results[sim_results['description'] == 'TTT/ACC'], x=0, hue='description', common_norm=False)
  plt.xscale('log')
  TTT_ACC.savefig(simulation_path + "/" + date_time_meta + "_0_TTT_ACC.png")
  plt.close()

  TTT_ACC = sns.displot(data=sim_results[sim_results['description'] == 'TTT/ACC'], x=1, hue='description', common_norm=False)
  plt.xscale('log')
  TTT_ACC.savefig(simulation_path + "/" + date_time_meta + "_1_TTT_ACC.png")
  plt.close()

  TTT_ACC = sns.displot(data=sim_results[sim_results['description'] == 'TTT/ACC'], x=2, hue='description', common_norm=False)
  plt.xscale('log')
  TTT_ACC.savefig(simulation_path + "/" + date_time_meta + "_2_TTT_ACC.png")
  plt.close()

  # generate and save distribution plots for with internal methionine
  TTT_immut_ACC = sns.displot(data=sim_results[sim_results['description'] == 'TTT/immutable/ACC'], x=0, hue='description', common_norm=False)
  plt.xscale('log')
  TTT_immut_ACC.savefig(simulation_path + "/" + date_time_meta + "_0_TTT_immutable_ACC.png")
  plt.close()

  TTT_immut_ACC = sns.displot(data=sim_results[sim_results['description'] == 'TTT/immutable/ACC'], x=1, hue='description', common_norm=False)
  plt.xscale('log')
  TTT_immut_ACC.savefig(simulation_path + "/" + date_time_meta + "_1_TTT_immutable_ACC.png")
  plt.close()

  TTT_immut_ACC = sns.displot(data=sim_results[sim_results['description'] == 'TTT/immutable/ACC'], x=2, hue='description', common_norm=False)
  plt.xscale('log')
  TTT_immut_ACC.savefig(simulation_path + "/" + date_time_meta + "_2_TTT_immutable_ACC.png")
  plt.close()
